In [1]:
# importing necessary libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.tools as tls
import plotly.io as pio
import os

from matplotlib.colors import ListedColormap
from plotly.offline import init_notebook_mode, iplot, plot
from IPython.display import Image, display


# settings
np.set_printoptions(threshold=np.nan)
pd.set_option('display.max_columns', None)
plt.tight_layout()
sns.set_style("whitegrid")
init_notebook_mode(connected=True)

%matplotlib inline
%config InlineBackend.figure_format = 'retina'


# creating a directory for images
if not os.path.exists('images'):
    os.mkdir('images')

color='#298DE1' # Highspot color
scale=2 # scale for saving images

In [13]:
# read the data

engage_stat_raw = pd.read_csv('./Content Engagement by Contact Status.csv',
                              skipinitialspace=True)  # raw data

engage_stat = engage_stat_raw[['ContactStatus', 'Item',
                               'Spot', 'Pitches', 'Views']]  # select main columns

engage_stat['V2P_Ratio'] = engage_stat['Views']/engage_stat['Pitches']

In [32]:
engage_stat['ContactStatus'].unique()

statuses = ['Suspect', 'Prospect', 'MQL', 'SAL',
            'TAL', 'SEL', 'TEL',  'SQL', 'Opp Contact']

top_viewed_names = np.chararray(
    (10, len(statuses)), itemsize=100, unicode=True)
top_viewed_pitches = np.empty([10, len(statuses)])
top_viewed_views = np.empty([10, len(statuses)])
top_viewed_v2p = np.empty([10, len(statuses)])


for status in statuses:
    i = statuses.index(status)
    top_viewed = engage_stat[engage_stat['ContactStatus'] == status].sort_values(
        by='Views', ascending=False)[:10]
    top_viewed_names[:, i] = top_viewed['Item'].values
    top_viewed_pitches[:, i] = top_viewed['Pitches'].values
    top_viewed_views[:, i] = top_viewed['Views'].values
    top_viewed_v2p[:, i] = top_viewed['V2P_Ratio'].values

chararray([['Highspot Product Overview (6p)',
            'Content List with Hero Banner',
            'Webinar: Building Your Content Mapping Plan',
            'SiriusDecisions: The Sales Enablement Range of Responsibilities Model',
            'Highspot Product Overview (6p)',
            'Content List with Hero Banner',
            'Highspot Product Overview (6p)',
            'Content List with Hero Banner',
            'Highspot Information Security'],
           ['Seattle Skyline Pitch Experience',
            'Highspot Product Overview (6p)',
            'Best Practices for Sales Playbooks Report',
            'Highspot Product Overview (6p)',
            'Content List with Hero Banner',
            'Highspot Product Overview (6p)',
            'Content List with Hero Banner', '#Love Pitch Experience',
            'Content List with Hero Banner'],
           ['Content List with Hero Banner',
            'Highspot Product Overview (6p)',
            'Sales Enablement Solution Re

In [50]:
# Add Periodic Table Data
symbol = top_viewed_names

element = top_viewed_pitches

atomic_mass = top_viewed_views

z = top_viewed_v2p

# Display element name and atomic mass on hover
hover=list(range(len(symbol)))
for x in range(len(symbol)):
    hover[x] = ['Pitches: ' + str(i) + '<br>' + 'Views: ' + str(j) for i, j in zip(element[x], atomic_mass[x])]

# Invert Matrices
symbol = symbol[::-1]
hover = hover[::-1]
z = z[::-1]

# Set Colorscale
colorscale='RdBu'

# Make Annotated Heatmap
pt = ff.create_annotated_heatmap(z, annotation_text=symbol, text=hover,
                                 colorscale=colorscale, font_colors=['black'], hoverinfo='text')
pt.layout.title = 'Periodic Table'

pt.layout.height=1500
pt.layout.width=3000

pt.layout.xaxis.showgrid=False
pt.layout.yaxis.showgrid=False
pt.layout.xaxis.showline=False
pt.layout.yaxis.showline=False
pt.layout.xaxis.zeroline=False
pt.layout.yaxis.zeroline=False


iplot(pt)

plot(pt)

'file:///Users/amin.derakhshi/Documents/Content Engagement/temp-plot.html'

In [43]:
help(ff.create_annotated_heatmap)

Help on function create_annotated_heatmap in module plotly.figure_factory._annotated_heatmap:

create_annotated_heatmap(z, x=None, y=None, annotation_text=None, colorscale='RdBu', font_colors=None, showscale=False, reversescale=False, **kwargs)
    BETA function that creates annotated heatmaps
    
    This function adds annotations to each cell of the heatmap.
    
    :param (list[list]|ndarray) z: z matrix to create heatmap.
    :param (list) x: x axis labels.
    :param (list) y: y axis labels.
    :param (list[list]|ndarray) annotation_text: Text strings for
        annotations. Should have the same dimensions as the z matrix. If no
        text is added, the values of the z matrix are annotated. Default =
        z matrix values.
    :param (list|str) colorscale: heatmap colorscale.
    :param (list) font_colors: List of two color strings: [min_text_color,
        max_text_color] where min_text_color is applied to annotations for
        heatmap values < (max_value - min_value)/2